In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_10%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,NaN,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,NaN,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,NaN,0.0
744,NaN,2.0,NaN,52.0,0.0
745,NaN,NaN,NaN,62.0,0.0
746,39.0,1.0,250.0,NaN,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              107
Frequency (times)                              78
Monetary (c.c. blood)                          94
Time (months)                                  95
whether he/she donated blood in March 2007      0
dtype: int64

In [6]:
df.isnull().sum().sum()

374

In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              14.304813
Frequency (times)                             10.427807
Monetary (c.c. blood)                         12.566845
Time (months)                                 12.700535
whether he/she donated blood in March 2007     0.000000
dtype: float64

In [8]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 374
persentase : 10.0


In [9]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [10]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,7)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
    
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [11]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [12]:
data = df.iloc[:, :-1]
df2 = df.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,0.0,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,0.0,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,0.0,0.0
744,0.0,2.0,0.0,52.0,0.0
745,0.0,0.0,0.0,62.0,0.0
746,39.0,1.0,250.0,0.0,0.0


In [13]:
euc_dist = {}

In [14]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,7)
dummy

Kolom : 1
[1.0, 2.0, 2.0, 4.0, 12.0, 14.0, 21.0]
min :  1.0
max :  21.0
mean :  8.0
fuzzy :  [(2.0, 0.14285714285714285), (4.0, 0.42857142857142855), (1.0, 0.0), (2.0, 0.14285714285714285), (14.0, 0.5384615384615384), (21.0, 0.0), (12.0, 0.6923076923076923)]
fuzzy :  None
pembilang :  18.13186813186813
penyebut :  1.945054945054945
rata-rata tertimbang :  9.322033898305085
 
[0.0, 4.0, 4.0, 9.0, 9.0, 11.0, 11.0]
min :  0.0
max :  11.0
mean :  6.857142857142857
fuzzy :  [(4.0, 0.5833333333333334), (0.0, 0.0), (4.0, 0.5833333333333334), (9.0, 0.48275862068965514), (11.0, 0.0), (9.0, 0.48275862068965514), (11.0, 0.0)]
fuzzy :  None
pembilang :  13.356321839080458
penyebut :  2.132183908045977
rata-rata tertimbang :  6.264150943396226
 
[1.0, 2.0, 3.0, 5.0, 6.0, 11.0, 11.0]
min :  1.0
max :  11.0
mean :  5.571428571428571
fuzzy :  [(11.0, 0.0), (1.0, 0.0), (5.0, 0.875), (6.0, 0.9210526315789473), (11.0, 0.0), (2.0, 0.21875), (3.0, 0.4375)]
fuzzy :  None
pembilang :  11.651315789473685
peny

[2.0, 2.0, 4.0, 4.0, 4.0, 4.0, 11.0]
min :  2.0
max :  11.0
mean :  4.428571428571429
fuzzy :  [(2.0, 0.0), (11.0, 0.0), (4.0, 0.8235294117647058), (4.0, 0.8235294117647058), (4.0, 0.8235294117647058), (2.0, 0.0), (4.0, 0.8235294117647058)]
fuzzy :  None
pembilang :  13.176470588235293
penyebut :  3.2941176470588234
rata-rata tertimbang :  4.0
 
[2.0, 2.0, 2.0, 2.0, 4.0, 4.0, 11.0]
min :  2.0
max :  11.0
mean :  3.857142857142857
fuzzy :  [(2.0, 0.0), (2.0, 0.0), (2.0, 0.0), (4.0, 0.9800000000000001), (2.0, 0.0), (11.0, 0.0), (4.0, 0.9800000000000001)]
fuzzy :  None
pembilang :  7.840000000000001
penyebut :  1.9600000000000002
rata-rata tertimbang :  4.0
 
[0.0, 0.0, 2.0, 11.0, 14.0, 16.0, 16.0]
min :  0.0
max :  16.0
mean :  8.428571428571429
fuzzy :  [(2.0, 0.23728813559322035), (11.0, 0.6603773584905661), (0.0, 0.0), (16.0, 0.0), (14.0, 0.2641509433962264), (0.0, 0.0), (16.0, 0.0)]
fuzzy :  None
pembilang :  11.436840422129837
penyebut :  1.1618164374800128
rata-rata tertimbang :  9

[0.0, 0.0, 2.0, 4.0, 6.0, 14.0, 21.0]
min :  0.0
max :  21.0
mean :  6.714285714285714
fuzzy :  [(0.0, 0.0), (6.0, 0.8936170212765957), (14.0, 0.49000000000000005), (21.0, 0.0), (0.0, 0.0), (4.0, 0.5957446808510638), (2.0, 0.2978723404255319)]
fuzzy :  None
pembilang :  15.200425531914895
penyebut :  2.277234042553191
rata-rata tertimbang :  6.674950948332245
 
[0.0, 0.0, 1.0, 2.0, 3.0, 4.0, 14.0]
min :  0.0
max :  14.0
mean :  3.4285714285714284
fuzzy :  [(0.0, 0.0), (1.0, 0.2916666666666667), (0.0, 0.0), (2.0, 0.5833333333333334), (14.0, 0.0), (4.0, 0.9459459459459459), (3.0, 0.875)]
fuzzy :  None
pembilang :  7.867117117117117
penyebut :  2.695945945945946
rata-rata tertimbang :  2.91812865497076
 
[0.0, 2.0, 4.0, 10.0, 11.0, 11.0, 11.0]
min :  0.0
max :  11.0
mean :  7.0
fuzzy :  [(0.0, 0.0), (11.0, 0.0), (10.0, 0.25), (11.0, 0.0), (11.0, 0.0), (2.0, 0.2857142857142857), (4.0, 0.5714285714285714)]
fuzzy :  None
pembilang :  5.357142857142857
penyebut :  1.1071428571428572
rata-rata

[0.0, 1.0, 4.751173708920188, 5.0, 6.0, 11.0, 11.0]
min :  0.0
max :  11.0
mean :  5.53588195841717
fuzzy :  [(0.0, 0.0), (11.0, 0.0), (1.0, 0.1806396898473467), (5.0, 0.9031984492367336), (4.751173708920188, 0.8582505451902107), (6.0, 0.9150607585614338), (11.0, 0.0)]
fuzzy :  None
pembilang :  14.264693913373765
penyebut :  2.8571494428357247
rata-rata tertimbang :  4.992631361702956
 
[0.0, 1.0, 2.0, 2.91812865497076, 3.0, 4.0, 14.0]
min :  0.0
max :  14.0
mean :  3.8454469507101083
fuzzy :  [(0.0, 0.0), (1.0, 0.26004779491635893), (2.91812865497076, 0.7588529220073865), (2.0, 0.5200955898327179), (4.0, 0.9847799259563965), (14.0, 0.0), (3.0, 0.7801433847490767)]
fuzzy :  None
pembilang :  9.794219289272657
penyebut :  3.3039196174619367
rata-rata tertimbang :  2.964424206178646
 
[0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 4.0]
min :  0.0
max :  4.0
mean :  2.0
fuzzy :  [(0.0, 0.0), (4.0, 0.0), (2.0, 1.0), (2.0, 1.0), (2.0, 1.0), (2.0, 1.0), (2.0, 1.0)]
fuzzy :  None
pembilang :  10.0
penyebut 

[0.0, 3.0, 8.0, 8.6158940397351, 11.0, 14.0, 14.0]
min :  0.0
max :  14.0
mean :  8.373699148533586
fuzzy :  [(0.0, 0.0), (8.0, 0.955372274319286), (14.0, 0.0), (3.0, 0.3582646028697322), (11.0, 0.5332100218597612), (8.6158940397351, 0.9569530855893728), (14.0, 0.0)]
fuzzy :  None
pembilang :  22.82808863005645
penyebut :  2.803799984638152
rata-rata tertimbang :  8.141839202200636
 
[0.0, 2.0, 4.0, 4.0, 14.0, 23.0, 23.0]
min :  0.0
max :  23.0
mean :  10.0
fuzzy :  [(0.0, 0.0), (14.0, 0.6923076923076923), (4.0, 0.4), (2.0, 0.2), (4.0, 0.4), (23.0, 0.0), (23.0, 0.0)]
fuzzy :  None
pembilang :  13.292307692307691
penyebut :  1.6923076923076925
rata-rata tertimbang :  7.854545454545453
 
[0.0, 0.0, 4.0, 15.751091703056769, 16.0, 23.0, 26.0]
min :  0.0
max :  26.0
mean :  12.107298814722396
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (4.0, 0.33037922506183015), (16.0, 0.7198024247867086), (26.0, 0.0), (15.751091703056769, 0.7377189043556354), (23.0, 0.2159407274360126)]
fuzzy :  None
pembilang :  2

...

[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.5714285714285714
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy : 

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.42857142857142855
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0), (1.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.5714285714285714
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :

[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.5714285714285714
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.5714285714285714
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0)]
fuzzy : 

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.14285714285714285
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.2857142857142857
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0), (1.0, 0.0)]
fuzzy :

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.000000,50.000000,8013.957597,98.000000,1.0
1,9.322034,13.000000,3250.000000,28.000000,1.0
2,1.000000,16.000000,4000.000000,70.766990,1.0
3,2.000000,20.000000,5000.000000,45.000000,1.0
4,1.000000,24.000000,6000.000000,77.000000,1.0
...,...,...,...,...,...
743,23.000000,2.000000,500.000000,25.796875,1.0
744,9.859402,2.000000,607.142857,52.000000,1.0
745,5.057438,7.356219,1946.374622,62.000000,1.0
746,39.000000,1.000000,250.000000,23.000000,1.0


In [15]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.0
mean,9.238380,5.514380,1352.483072,34.088808,1.0
std,7.773023,5.836637,1397.924956,24.391986,0.0
min,1.000000,1.000000,250.000000,2.000000,1.0
25%,3.000000,2.000000,500.000000,15.000000,1.0
50%,7.184411,4.000000,1000.000000,28.000000,1.0
75%,14.000000,7.000000,1750.000000,50.000000,1.0
max,74.000000,50.000000,11500.000000,98.000000,1.0


# Klasifikasi

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [17]:
x = dummy.values
y = df.iloc[:, -1].values

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=94)

In [19]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [20]:
classifier = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[135  15]
 [ 22  15]]
0.8021390374331551


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [21]:
classifier = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[137  13]
 [ 22  15]]
0.8128342245989305


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [22]:
classifier = KNeighborsClassifier(n_neighbors=7, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[138  12]
 [ 26  11]]
0.7967914438502673


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [23]:
classifier = KNeighborsClassifier(n_neighbors=9, metric='euclidean')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[139  11]
 [ 28   9]]
0.7914438502673797


C:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
